In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Lambda, Flatten, Dense, Dropout, Concatenate, Bidirectional, LSTM
from tensorflow.keras.models import Model as KerasModel

from coremdlr.networks.util import unstack_rows, stack_rows, make_backbone, make_dense_layers
from coremdlr.ops import bilinear_pooling



def bilinear_cnn(num_classes,
                 input_shape,
                 backbone_cnn=None,
                 fB=None,
                 conv1x1=None,
                 dense_layers=[],
                 dropout_rate=None,
                 apply_rowwise=False,
                 lstm_features=None):
    '''Combine two feature extracting CNNs into single Model with bilinear_pooling + FC layers.
       fA and fB should output 4D tensors of equal shape, except (optionally) in # of channels.

    Parameters
    ----------
    fA : KerasModel or str
        Feature network A. Should output features (N, H, W, cA).
        If str, loads the corresponding ImageNet model from `keras.applications`.
    fB : KerasModel or str, optional
        Feature network B. Should output features (N, H, W, cB).
        If str, loads the corresponding ImageNet model from `keras.applications`.
        If `None`, will return symmetric BCNN using fA.
    num_classes : int
            Number of classes for softmax output layer
    input_shape : tuple of int
        Shape of input images. Must be compatible with fA.input & fB.input.
    conv1x1 : int or iterable(int), optional
        Add a 1x1 conv to reduce number of channels in (fA, fB) to some value(s).
        If iterable, must be length 2; values then mapped to (fA, fB).
    dense_layers : iterable of int, optional
        Sizes for additional Dense layers between bilinear vector and softmax. Default=[].
    dropout_rate: float, optional
        Specify a dropout rate for Dense layers

    Returns
    -------
    B-CNN : KerasModel
        Single bilinear CNN composed from fA & fB (asymmetric) or fA with itself (symmetric)
    '''
    assert backbone_cnn is not None
    fA = make_backbone(backbone_cnn, input_shape)
    fB = make_backbone(fB, input_shape)

    input_image = Input(shape=input_shape)

    outA = fA(input_image)
    if fB is None:
        outB = outA             # symmetric B-CNN
    else:
        outB = fB(input_image)  # asymmetric B-CNN

    if isinstance(conv1x1, int):
        outA = Conv2D(conv1x1, (1,1), name='reduce_A')(outA)
        outB = Conv2D(conv1x1, (1,1), name='reduce_B')(outB)
    elif hasattr(conv1x1, '__iter__'):
        assert len(conv1x1) == 2, 'if iterable, conv1x1 must have length of 2'
        outA = Conv2D(conv1x1[0], (1,1), name='reduce_A')(outA)
        outB = Conv2D(conv1x1[1], (1,1), name='reduce_B')(outB)
        
    bilinear_pooling_layer = Lambda(bilinear_pooling, name='bilinear_pooling')

    if apply_rowwise:
        rowsA = unstack_rows(outA)
        rowsB = unstack_rows(outB)
        x = stack_rows([bilinear_pooling_layer([rA, rB]) for rA, rB in zip(rowsA, rowsB)])
        if lstm_features is not None:
            x = Bidirectional(LSTM(lstm_features, return_sequences=True))(x)
    else:
        x = Lambda(bilinear_pooling, name='bilinear_pooling')([outA, outB])

    x = make_dense_layers(dense_layers, dropout=dropout_rate)(x)

    pred = Dense(num_classes, activation='softmax')(x)

    model = KerasModel(inputs=input_image, outputs=pred)

    return model

Using TensorFlow backend.


In [2]:
bcnn = bilinear_cnn(5, (None,800,3), backbone_cnn='vgg16')
bcnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 800, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, None, 25, 512 14714688    input_2[0][0]                    
__________________________________________________________________________________________________
bilinear_pooling (Lambda)       (None, 262144)       0           vgg16[1][0]                      
                                                                 vgg16[1][0]                      
__________________________________________________________________________________________________
dense (Dense)                   (None, 5)            1310725     bilinear_pooling[0][0]           
Total para

In [3]:
bcnn = bilinear_cnn(5, (None,800,3), backbone_cnn='vgg16', 
                    conv1x1=64, apply_rowwise=True, lstm_features=64)
bcnn.summary()

ValueError: Cannot infer num from shape (?, ?, 25, 64)

640

In [ ]:
from tensorflow.keras.layers import Bidirectional, TimeDistributed, LSTM

x = bcnn.get_layer('stack_rows').output

In [ ]:
x

In [ ]:
Bidirectional(LSTM(64, return_sequences=True))(x)